In [1]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)


Azure ML SDK Version:  1.3.0


In [ ]:
!pip install --upgrade azureml-sdk

In [20]:
!pip install --upgrade scikit-learn

     |████████████████████████████████| 7.1MB 7.1MB/s eta 0:00:01
ERROR: azureml-train-automl-runtime 1.0.85.5 has requirement azureml-automl-core==1.0.85.*, but you'll have azureml-automl-core 1.3.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.0.85.5 has requirement azureml-core==1.0.85.*, but you'll have azureml-core 1.3.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.0.85.5 has requirement azureml-dataprep[fuse,pandas]<1.2.0a,>=1.1.37a, but you'll have azureml-dataprep 1.4.3 which is incompatible.
ERROR: azureml-train-automl-runtime 1.0.85.5 has requirement azureml-pipeline-core==1.0.85.*, but you'll have azureml-pipeline-core 1.3.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.0.85.5 has requirement azureml-telemetry==1.0.85.*, but you'll have azureml-telemetry 1.3.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.0.85.5 has requirement azureml-train-automl-client==1.0.85.*, but you'll have azureml-train-automl-client 1.3.0

In [22]:
# sckit learn version
import sklearn
print(sklearn.__version__)

0.20.3


## Initialize workspace

In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()

print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')


sandboxaml	westeurope	rg-sandbox	westeurope


## Create an experiment

In [3]:
experiment_name = 'diabetes_exp'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

exp


Name,Workspace,Report Page,Docs Page
diabetes_exp,sandboxaml,Link to Azure Machine Learning studio,Link to Documentation


## Upload dataset

In [4]:
from azureml.core import Dataset

diabetes_dataset = Dataset.get_by_name(ws, name='diabetes')


In [5]:
diabetes = diabetes_dataset.to_pandas_dataframe().drop("Path", axis=1)

In [6]:
diabetes.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [7]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
AGE    442 non-null int64
SEX    442 non-null int64
BMI    442 non-null float64
BP     442 non-null float64
S1     442 non-null int64
S2     442 non-null float64
S3     442 non-null float64
S4     442 non-null float64
S5     442 non-null float64
S6     442 non-null int64
Y      442 non-null int64
dtypes: float64(6), int64(5)
memory usage: 38.1 KB


In [8]:
diabetes.describe()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [9]:
features_names = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
target = "Y"

## Create a (remote) target compute

In [ ]:
# Compute target creation

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "myComputeCluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print(" Cluster already exists")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=0, max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=30)

In [10]:
# Retrieve existing compute target

from azureml.core.compute import ComputeTarget

compute_target_name = "computetarget"
compute_target = ComputeTarget(workspace=ws, name=compute_target_name)

print(compute_target.provisioning_state)

Succeeded


## Train the model

### Train the model on a remote target compute

In [11]:
import os
script_folder = os.path.join(os.getcwd(), "remote_train")
os.makedirs(script_folder, exist_ok=True)

In [21]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.externals import joblib
import pickle

from azureml.core import Run
from azureml.core import Dataset
from utils import load_data

from azureml.core.authentication import ServicePrincipalAuthentication
sp = ServicePrincipalAuthentication(tenant_id=os.environ.get('AML_TENANT_ID'),
                               service_principal_id=os.environ.get('AML_PRINCIPAL_ID'),
                                    service_principal_password=os.environ.get('AML_PRINCIPAL_PASS'))

from azureml.core import Workspace
ws = Workspace.get(name="sandboxaml",
                   resource_group="rg-sandbox",
                   subscription_id=os.environ.get('SUBSCRIPTION_ID'),
                   auth=sp)
ws.get_details()
print("Found workspace {} at location {}".format(ws.name, ws.location))

parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg', default=0.5, help='regularization strength')
args = parser.parse_args()

# load train and test set into numpy arrays
diabetes_dataset = Dataset.get_by_name(ws, name='diabetes')
diabetes = diabetes_dataset.to_pandas_dataframe().drop("Path", axis=1)
target = "Y"
X = diabetes.drop(target, axis=1)
y = diabetes["Y"].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# get hold of the current run
run = Run.get_context()

print('Train a Ridge regression model with regularization strength of', args.reg)
model = Ridge(alpha=args.reg, solver="auto", random_state=42)
model.fit(X_train, y_train)

print('Predict the test set')
y_hat = model.predict(X_test)

# calculate score on the prediction
score = model.score(X_test, y_test)
print('Score is ', score)

run.log('regularization strength', np.float(args.reg))
run.log('score', np.float(score))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_reg_remote_model.pkl')

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/sandboxvm/code/users/paul.peton/Diabetes/remote_train/train.py


In [ ]:
import shutil
shutil.copy('utils.py', script_folder)

In [13]:
# Set up the (compute target) environnement

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment("diabetes_remote_env")

env.docker.enabled = True
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn',
                                                                          'pandas',
                                                                          'numpy',
                                                                          'joblib',                                                                        
                                                                          'matplotlib'
                                                                         ])
env.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")

env.python.conda_dependencies.save_to_file(".", "diabetes_env.yml")


'diabetes_env.yml'

In [14]:
from azureml.train.estimator import Estimator

script_params = {
    '--regularization': 0.5
}

est = Estimator(source_directory=script_folder,
              script_params=script_params,
              compute_target=compute_target,
              environment_definition=env,
              entry_script='train.py')


In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run = exp.submit(config=est)
run

In [ ]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True) 

Pipeline

In [23]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

pipeline_steps = [ PythonScriptStep(
    script_name="train.py",
    #arguments=["--input", input_data, "--output", output_data],
    #inputs=[input_data],
    #outputs=[output_data],
    compute_target=compute_target,
    #runconfig=???,
    source_directory="remote_train"
) ]


In [24]:
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
pipeline_run = exp.submit(pipeline, regenerate_outputs=False)

RunDetails(pipeline_run).show()


Created step train.py [c38b1612][a292cc06-fcce-4f17-8551-1ea28c70d060], (This step will run and generate new outputs)
Submitted PipelineRun a91e704c-910d-4835-8b89-8552150ad216
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/diabetes_exp/runs/a91e704c-910d-4835-8b89-8552150ad216?wsid=/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-sandbox/workspaces/sandboxaml


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [ ]:
# specify show_output to True for a verbose log
pipeline_run.wait_for_completion(show_output=True) 

PipelineRunId: a91e704c-910d-4835-8b89-8552150ad216
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/diabetes_exp/runs/a91e704c-910d-4835-8b89-8552150ad216?wsid=/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-sandbox/workspaces/sandboxaml
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: ddb86316-f195-4a38-9f77-861835f1cbe3
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/diabetes_exp/runs/ddb86316-f195-4a38-9f77-861835f1cbe3?wsid=/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-sandbox/workspaces/sandboxaml
StepRun( train.py ) Status: NotStarted


Scheduler

In [ ]:
from azureml.pipeline.core import Pipeline, PublishedPipeline

published_pipelines = PublishedPipeline.list(ws)

for published_pipeline in  published_pipelines:
    print(f"{published_pipeline.name},'{published_pipeline.id}'")


In [ ]:
from azureml.pipeline.core import Schedule, ScheduleRecurrence

recurrence = ScheduleRecurrence(frequency="Minute", interval=15)
recurring_schedule = Schedule.create(ws, name="MyRecurringSchedule", 
                            description="Based on time",
                            pipeline_id=pipeline_id, 
                            experiment_name=experiment_name, 
                            recurrence=recurrence)

In [ ]:
pipeline = PublishedPipeline.get(ws, id=pipeline_id)
pipeline.disable()


## Register the model

In [ ]:
print(run.get_file_names())

In [ ]:
# register model 
model = run.register_model(model_name='diabetes_ridge_regression', model_path='outputs/diabetes_reg_remote_model.pkl')

print(model.name, model.id, model.version, sep='\t')

## Create a scoring script

In [ ]:
%%writefile score.py

import joblib
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'diabetes_regression_model.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    model = joblib.load(model_path)


# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and validates the incoming payload against
# the example input you provide here. This will also generate a Swagger
# API document for your web service.
@input_schema('data', NumpyParameterType(np.array([[59, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87]])))
@output_schema(NumpyParameterType(np.array([151.000])))
def run(data):
    # Use the model object loaded by init().
    result = model.predict(data)

    # You can return any JSON-serializable object.
    return result.tolist()

## Define the (inference) environement

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])


## Define a inference configuration

In [ ]:
from azureml.core.model import InferenceConfig


inference_config = InferenceConfig(entry_script='score.py', environment=environment)

## Deploy in a custom environment

In [ ]:
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'diabetes-custom-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)


In [ ]:
# reload the model

from azureml.core import Webservice


service_name = 'diabetes-remoteTrain-service'
service = Webservice(ws, service_name)

print(service.name, service.scoring_uri, sep='\t')


## Test the service

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [59, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87],
        [69, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87]
    ],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)


In [ ]:
service.delete()